# --- 30 Days of Pandas Challenge: Solutions with concepts review ---

In [2]:
import pandas as pd

# Filtering

In [6]:
# 595: or-filtering

data = [['Afghanistan', 'Asia', 652230, 25500100, 20343000000], ['Albania', 'Europe', 28748, 2831741, 12960000000], ['Algeria', 'Africa', 2381741, 37100000, 188681000000], ['Andorra', 'Europe', 468, 78115, 3712000000], ['Angola', 'Africa', 1246700, 20609294, 100990000000]]
world = pd.DataFrame(data, columns=['name', 'continent', 'area', 'population', 'gdp']).astype({'name':'object', 'continent':'object', 'area':'Int64', 'population':'Int64', 'gdp':'Int64'})

def big_countries(world: pd.DataFrame) -> pd.DataFrame:
    big_countries = world[(world["area"] >= 3000000) | (world["population"] >= 25000000)]
    return big_countries[["name","population","area"]]

big_countries(world)

,name,population,area
0,Afghanistan,25500100,652230
2,Algeria,37100000,2381741


In [7]:
# 1757: and-filtering

def find_products(products: pd.DataFrame) -> pd.DataFrame:
    return products[(products["low_fats"] == "Y") & (products["recyclable"] == "Y")][["product_id"]]

In [8]:
# 183: isin filtering, negation

def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    does_not_order = ~customers["id"].isin(orders["customerId"])
    return customers[does_not_order][["name"]].rename(columns={"name":"Customers"})

In [14]:
# 1148: .drop_duplicates(), .sort_values()

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    self_views = views[views["author_id"] == views["viewer_id"]]
    unique_self_views = self_views.drop_duplicates(subset=["author_id", "viewer_id"])

    return unique_self_views[["author_id"]].rename(columns={"author_id": "id"}).sort_values("id")

# String Methods

In [15]:
# 1683: .len()
def invalid_tweets(tweets: pd.DataFrame) -> pd.DataFrame:
    return tweets[tweets["content"].str.len() > 15][["tweet_id"]]

In [16]:
# 1873: .startswith(), .loc[mask, "col"]
def calculate_special_bonus(employees: pd.DataFrame) -> pd.DataFrame:
    employees["bonus"] = 0
    
    # access the bonus field of each row that deserves a bonus
    # loc is efficient to doing df[deserves_bonus]["bonus"], where an intermiate df gets produced. I guess .apply() is still nice too
    deserves_bonus = ~(employees["name"].str.startswith("M")) & (employees["employee_id"] % 2 == 1)
    employees.loc[deserves_bonus, "bonus"] = employees["salary"]
    return employees[["employee_id", "bonus"]].sort_values("employee_id", ascending=True)

In [17]:
# 1667: .lower(), .capitalize()
def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    users["name"] = users["name"].str.lower().str.capitalize()
    return users.sort_values("user_id", ascending=True)

In [18]:
# 1517: .apply() for more particular string conditions, no regex
def valid_prefix(s: str) -> bool:
    starts_with_letter = s[0].isalpha()
    if not starts_with_letter:
        return False
    
    valid_special_chars = set(".-_")
    for char in s:
        if (char in valid_special_chars) or char.isalpha() or char.isdigit():
            continue
        return False
    
    return True

def valid_emails(users: pd.DataFrame) -> pd.DataFrame:
    valid_prefix_mask = users["mail"].str.rsplit("@", n=1).str[0].apply(valid_prefix)
    valid_domain_mask = users["mail"].str.endswith("@leetcode.com")
    return users[valid_prefix_mask & valid_domain_mask]

In [19]:
# 1527: regex
def find_patients(patients: pd.DataFrame) -> pd.DataFrame:
    #has_diab1 = patients["conditions"].str.split(' ').apply(lambda l: pd.Series(l).str.startswith("DIAB1").any())
    has_diab1 = patients["conditions"].str.contains(r"\bDIAB1\w*\b")
    return patients[has_diab1]

# Data Manipulation

In [20]:
# 177: pairing .sort_values() with .iloc[] to access nth highest entry
def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    # remove duplicate salaries to avoid ranking two duplicate salaries different rankings
    employee = employee.drop_duplicates("salary")

    if N > len(employee):
        return pd.DataFrame({'Nth Highest Salary': [None]})

    nth_highest_salary = employee.sort_values("salary", ascending=False)["salary"].iloc[N-1]
    return pd.DataFrame({'Nth Highest Salary': [nth_highest_salary]})

In [3]:
# 176: Jokes on you, we can use .nlargest() to return the n rows containing the nlargest. then use .iloc[] to select the desired 2nd highest
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    employee = employee.drop_duplicates("salary")

    if len(employee) < 2:
        return pd.DataFrame({"SecondHighestSalary": [None]})
    
    # nlargest returns the rows containing the 1st and 2nd largest salaries
    second_highest_salary = employee.nlargest(2, "salary").iloc[-1]["salary"]
    return pd.DataFrame({"SecondHighestSalary": [second_highest_salary]})

In [15]:
#184: .groupby() and .transform() to attribute group statistics to each entry in original df. could also have been accomplished with a merge as well after groupby.  
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    employee["dept_highest_salary"] = employee.groupby("departmentId")["salary"].transform("max")
    
    is_highest_salary = employee["salary"] == employee["dept_highest_salary"]

    employees_w_highest_salaries = pd.merge(employee[is_highest_salary], department, left_on="departmentId", right_on="id").rename(columns={
        "name_y": "Department",
        "name_x": "Employee",
        "salary": "Salary"
        })

    return employees_w_highest_salaries[["Department", "Employee", "Salary"]]

In [6]:
# 178: .rank() and using its "method" parameter to deal with tiebreakers
def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    scores = scores.sort_values("score", ascending=False)
    scores["rank"] = scores["score"].rank(ascending=False, method="dense")
    return scores.drop(columns=["id"])

In [7]:
# 196: using the "inplace" parameter, .drop_duplicates() with "keep" parameter to deal with tiebreakers
# Modify Person in place
def delete_duplicate_emails(person: pd.DataFrame) -> None:
    person.sort_values("id", ascending=True, inplace=True)
    person.drop_duplicates(subset=["email"], keep="first", inplace=True) # "first" to keep the row with smallest id

In [8]:
# 1795: .melt() to convert wide data to long data, .dropna() to drop rows contaning None
def rearrange_products_table(products: pd.DataFrame) -> pd.DataFrame:
    # id_vars: the pivot column, 
    # var_name: the name of the new column holding the "wide" columns, 
    # value_name: since you're migrating the wide columns to its own column under var_name, this value_name will be the name of the new column name for the values. 
    products_long = products.melt(id_vars="product_id", var_name="store", value_name="price")
    products_long_nonulls = products_long.dropna()
    return products_long_nonulls

# Statistics

In [9]:
# 2082: .nunique() to count number of distinct entries
def count_rich_customers(store: pd.DataFrame) -> pd.DataFrame:
    filtered_store = store[store["amount"] > 500]
    n_rich_customers = filtered_store["customer_id"].nunique()
    
    return pd.DataFrame({"rich_count": [n_rich_customers]})

In [10]:
# 1173: calculating percentages, using .shape[0] to access number of rows 
def food_delivery(delivery: pd.DataFrame) -> pd.DataFrame:
    is_same_day = delivery["order_date"] == delivery["customer_pref_delivery_date"]
    prop_intermediate = round(100 * delivery[is_same_day].shape[0] / delivery.shape[0], 2)
    return pd.DataFrame({"immediate_percentage": [prop_intermediate]})

In [11]:
# 1907: classification, no new methods
def count_salary_categories(accounts: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({
        "category": ["Low Salary", "Average Salary", "High Salary"],
        "accounts_count": [
            accounts[ accounts["income"] < 20000 ].shape[0],
            accounts[ (20000 <= accounts["income"]) & (accounts["income"] <= 50000) ].shape[0],
            accounts[ 50000 < accounts["income"] ].shape[0]
        ]
    })

# Data Aggregation

In [12]:
# 1741: .groupby() for grouping, and GroupBy.sum() for computing the sum of each group (preceded by desired columns to operate on), .reset_index() to "flatten" the resulting statistics df
def total_time(employees: pd.DataFrame) -> pd.DataFrame:
    employees["total_time"] = employees["out_time"] - employees["in_time"]
    employees_total_times_by_date = employees.groupby(["event_day", "emp_id"])["total_time"].sum().reset_index()
    
    return employees_total_times_by_date.rename(columns={"event_day":"day"})

In [13]:
# 511: Groupby.min()
def game_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    # doing ["event_date"] makes it so only columns player_id (the by) and event_date (to be renamed first_login) are present
    first_logins = activity.groupby(by="player_id")["event_date"].min().reset_index().rename(columns={"event_date": "first_login"})
    return first_logins

In [14]:
# 2356: Groupby.nunique() 
def count_unique_subjects(teacher: pd.DataFrame) -> pd.DataFrame:
    # similarly to Groupby.min(), ["subject_id"].nunique() here leaves only the "teacher_id" (the by) and "subject_id" (to be renamed cnt) columns in the resulting df
    subject_counts = teacher.groupby(by=["teacher_id"])["subject_id"].nunique().reset_index().rename(columns={"subject_id": "cnt"})
    return subject_counts

In [16]:
# 596: Groupby.count() counts only non-null entries. use for when unconcerned about duplicates
def find_classes(courses: pd.DataFrame) -> pd.DataFrame:
    # Groupby.count() counts only non-null entries
    course_counts = courses.groupby(by=["class"])["student"].count().reset_index().rename(columns={"student": "n_students"})
    return course_counts[course_counts["n_students"] >= 5][["class"]]

In [17]:
# 586: going with the trend in this section, there's nothing really special here. but we can alternatively use DataFrame.mode() which is an inbuilt function for this exact purpose -- finding the entry with 
def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    order_counts = orders.groupby("customer_number")["order_number"].count().reset_index().rename(columns={"order_number": "n_orders"})
    max_orders_by_customer = order_counts["n_orders"].max()
    customer_w_most_orders = order_counts[order_counts["n_orders"] == max_orders_by_customer]
    return customer_w_most_orders[["customer_number"]]

def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    # here, .mode() returned a Series
    return orders["customer_number"].mode().to_frame()

In [18]:
# 1484: .agg() to apply function(s) to each column of each group. in this case, we apply nunique and a string concatenation on the lone column in each group "product", generating two new columns in the resulting df, one for nunique and one for the string concatenation. 
def categorize_products(activities: pd.DataFrame) -> pd.DataFrame:
    # note that agg runs each function on each group for each column. so we extract "product" column at the end
    ret_df = activities.groupby("sell_date").agg(["nunique", lambda col: ",".join(sorted(col.unique()))])["product"].reset_index()
    
    return ret_df.rename(columns={"nunique": "num_sold", "<lambda_0>": "products"})

In [19]:
#1693: .agg() with particular functions for each column in each group. 
def daily_leads_and_partners(daily_sales: pd.DataFrame) -> pd.DataFrame:
    # alternatively, use .nunique() instead of .agg(), then .rename()
    return daily_sales.groupby(by=["date_id", "make_name"]).agg(unique_leads=("lead_id", "nunique"), unique_partners=("partner_id", "nunique")).reset_index()

In [20]:
# 1050: 
# (this is classified under Data Integration, but it fits more in Aggregation imho)
def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    # This also works. .filter(function) filters all elements of each group and aggregates it all back into a dataframe. 
    #return actor_director.groupby(["actor_id", "director_id"]).filter(lambda df: df.shape[0] >= 3).drop(columns=["timestamp"]).drop_duplicates()
    
    # keep in mind .agg returns a df (indexed up) where all groups are visible along with their rows 
    num_coops = actor_director.groupby(["actor_id", "director_id"]).agg(count=("timestamp", "count")).reset_index()
    coops_3_or_more = num_coops[num_coops["count"] >= 3]
    return coops_3_or_more.drop(columns=["count"])

# Data Integration

In [ ]:
# 1378: 